In [1]:
import findspark
from pyspark.sql.session import SparkSession
from pyspark import SparkContext, SparkConf

In [8]:
pip install mysql-connector-python

     |████████████████████████████████| 19.4 MB 3.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


24/06/08 00:24:37 WARN DataStreamer: Exception for BP-2032145477-10.5.0.2-1717805737242:blk_1073741831_1007
java.io.EOFException: Unexpected EOF while trying to read response from server
	at org.apache.hadoop.hdfs.protocolPB.PBHelperClient.vintPrefixed(PBHelperClient.java:521)
	at org.apache.hadoop.hdfs.protocol.datatransfer.PipelineAck.readFields(PipelineAck.java:213)
	at org.apache.hadoop.hdfs.DataStreamer$ResponseProcessor.run(DataStreamer.java:1137)
24/06/08 00:24:37 WARN DataStreamer: Error Recovery for BP-2032145477-10.5.0.2-1717805737242:blk_1073741831_1007 in pipeline [DatanodeInfoWithStorage[10.5.0.4:9866,DS-a92dd940-3048-48c3-a1ee-90c7e2122fd1,DISK], DatanodeInfoWithStorage[10.5.0.3:9866,DS-186516c4-0942-4a77-acc4-ac7658eea6df,DISK]]: datanode 0(DatanodeInfoWithStorage[10.5.0.4:9866,DS-a92dd940-3048-48c3-a1ee-90c7e2122fd1,DISK]) is bad.
24/06/08 00:24:37 ERROR TransportClient: Failed to send RPC RPC 6978772683075901008 to /10.5.0.3:51808: io.netty.channel.StacklessClosedChann

In [2]:
# Inicializando Spark
findspark.init("/usr/spark-3.5.1/")

In [3]:
spark = (
    SparkSession.builder.appName("sparksubmit_test_app")
    .config("spark.sql.warehouse.dir", "hdfs:///user/hive/warehouse")
    .config("spark.sql.catalogImplementation", "hive")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/08 00:29:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/08 00:29:46 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
# criar um contexto de sessão do spark (cria um "programa")
sc = SparkContext.getOrCreate()

In [5]:
def main():

    df = spark.read.text("hdfs://spark-master:9000/datasets/")  # noqa: F841

    # variável recebe o caminho que aponta para uma arquivo de texto
    file_path = "/datasets/*.txt"
    # leitura do arquivo de texto pelo programa spark
    words = sc.textFile(f"{file_path}").flatMap(lambda line: line.split(" "))
    # contagem de palavras utilizando a sintaxe facilitada do pyspark
    wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
    # salvando arquivo com resultado da execução
    wordCounts.saveAsTextFile(f"{'/'.join(file_path.split('/')[:-1])}/word_count")
   
    print(wordCounts.count())
    print(wordCounts.countApproxDistinct())

In [7]:
if __name__ == "__main__":
    main()

66676


68240


In [ ]:
spark.stop()